In [98]:
import random
import math
import pylab as pl
import numpy as np
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.base import BaseEstimator
from sklearn.datasets import  fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from scipy.spatial import distance
from scipy.stats import mode
from scipy.spatial import distance_matrix
import sklearn
from sklearn.datasets import  fetch_openml
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold

In [99]:
class MyKNeighborsClassifier(BaseEstimator):
    
    def __init__(self, n_neighbors, algorithm):
        self.n_neighbors = n_neighbors
        self.algorithm = algorithm
        self.data = np.array([])
        self.labels = np.array([],dtype = int)
        self.numberOfClasses = 0
        pass
    
    def fit(self, X, y):
        if self.algorithm == 'brute':
            self.data = np.array(X)
            self.labels = np.array(y)
            self.numberOfClasses = len(set(y))
        else:
            self.data = sklearn.neighbors.KDTree(X)
            self.labels = np.array(y)
        pass
    
    def predict(self, X):
        if (self.algorithm == 'brute'):
            idx = np.array([])
            Dist = distance_matrix(X_test, my_clf.data)
            idx = np.argpartition(Dist, self.n_neighbors, axis = 1)[:,:self.n_neighbors]
            lbl = np.array([self.labels]*len(X)).reshape(len(X),-1)
            sublab = np.take_along_axis(lbl, idx, axis = 1)
            Labels = mode(sublab,1)[0]
        else:
            dist, lb = self.data.query(X, self.n_neighbors)
            lbl = np.array([self.labels]*len(X)).reshape(len(X),-1)
            sublab = np.take_along_axis(lbl, lb, axis = 1)
            Labels = mode(sublab,1)[0]
        return Labels
        pass

In [100]:
iris = datasets.load_iris()

In [101]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)


In [102]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [103]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [104]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

In [105]:
%time clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 893 µs


KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [106]:
%time my_clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 83.7 µs


In [107]:
%time clf.predict(X_test)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 2.94 ms


array([2, 1, 0, 1, 1, 2, 1, 0, 0, 1, 2, 0, 0, 1, 1])

In [108]:
%time my_clf.predict(X_test)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 2.44 ms


array([[2],
       [1],
       [0],
       [1],
       [1],
       [2],
       [1],
       [0],
       [0],
       [1],
       [2],
       [0],
       [0],
       [1],
       [1]])

In [31]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [32]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [33]:
%time clf.fit(X_train, y_train)

CPU times: user 15.6 ms, sys: 0 ns, total: 15.6 ms
Wall time: 526 µs


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [34]:
%time my_clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 203 µs


In [35]:
%time clf.predict(X_test)


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 2.26 ms


array([1, 0, 1, 0, 2, 2, 2, 1, 2, 2, 0, 1, 0, 1, 0])

In [36]:
%time my_clf.predict(X_test)


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 2.01 ms


array([[1],
       [0],
       [1],
       [0],
       [2],
       [2],
       [2],
       [1],
       [2],
       [2],
       [0],
       [1],
       [0],
       [1],
       [0]])

In [37]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

In [109]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [110]:
data = newsgroups['data']
target = newsgroups['target']

In [116]:
CountV=CountVectorizer (min_df=100,max_df=0.8,stop_words='english')
X_train_count=CountV.fit_transform(data)


In [117]:
X_train_count.shape

(11314, 1336)

In [118]:
X_train_fold = KFold(n_splits=3)

In [119]:
X_train_fold.get_n_splits(X_train_count, target)

3

In [124]:
for train_index, test_index in X_train_fold.split(X_train_count):
    my_clf.fit(X_train_count[train_index], target[train_index])
    print(X_train_count[test_index].shape)
    my_clf_pred = my_clf.predict(X_train_count[test_index])
    print(accuracy_score(target[test_index], my_clf_pred))

(3772, 1336)


ValueError: not enough values to unpack (expected 2, got 0)